In [1]:
import pybullet as p
import time
from time import sleep
import pybullet_data   #we use this for getting access in pybullet urdf (Universal robot description file) and import them
import numpy as np
import math
import os  #provide function to the interaction with operating system

In [2]:
import motorController  #these are the file which have some necessary physics behind the picture which we will in 
import walkGenerator    # front end like we will see the stable walking of biped in frontend but what is going on
                        #in backend of the motorController we will it in these files

In [3]:
#for controlling the motor for reducing the error we will close loop control system. so here we are using pd contro
# controller for it. 
# PD controller is (propotional derivative controller we use it when the chance of the external disturbance is less
#or neglegible we will use it. If there is the external disturbance is also there then we will use pid controller).

In [4]:
# constant parameter of pd controller
motor_kp = 0.5  # PD controlle is bad controller, because controller should be generic but here we are tunning 
motor_kd = 0.5   # the kp and kd parameter other wise it is good in case of no external disturbance.
motor_torque = 2  # torque of motor give how much it will rotate the motor. unit is Newton-meter. 
motor_max_velocity = 10.0 #motor will rotate with this speed

In [5]:
fixedTimeStep = 1. / 1000
numSolverIterations = 200

In [6]:
#physic set of the simulation environment.
physicsClient = p.connect(p.GUI)
p.setTimeStep(timeStep=fixedTimeStep, physicsClientId=physicsClient)
p.setPhysicsEngineParameter(numSolverIterations = numSolverIterations)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

In [7]:
p.setGravity(0, 0, 0)
p.resetDebugVisualizerCamera(cameraDistance=1, cameraYaw=10, cameraPitch=-5, cameraTargetPosition=[0.3, 0.5, 0.1], physicsClientId=physicsClient)
planeId = p.loadSDF('stadium.sdf')  # can load p.loadURDF('samurai.urdf') or p.loadURDF('plane.urdf')

In [8]:
#add the biped robot at the plane.
robot = p.loadURDF('humanoid_leg_12dof.8.urdf', [0, 0, 0.31],
                  p.getQuaternionFromEuler([0, 0, 0]), useFixedBase=False)
controller = motorController.MotorController(robot, physicsClient, fixedTimeStep, motor_kp, motor_kd, motor_torque, motor_max_velocity)


In [11]:
walk = walkGenerator.WalkGenerator()
walk.setWalkParameter(bodyMovePoint=8,
                     legMovePoint=8,
                     height=50,
                     stride=90,
                     sit=40,
                     swayBody=30,
                     swayFoot=0,
                     bodyPositionForwardPlus=5,
                     swayShift=3,
                     liftPush=0.4,
                     landPull=0.6,
                     timeStep=0.06,
                     damping=0.0,
                     incline=0.0)
walk.generate()
walk.inverseKinematicsAll()

In [12]:
actionTime = walk._timeStep
p.setGravity(0, 0, -9.8)
p.setRealTimeSimulation(0)
controller.setMotorsAngleInFixedTimestep(walk.walkAnglesStartRight[0], 1, 0)

In [13]:
waitTime = 1
repeatTime = int(waitTime / fixedTimeStep)
for _ in range(repeatTime):
    p.stepSimulation()
p.setGravity(0, 0, -9.8)

In [17]:
for i in range(np.size(walk.walkAnglesStartRight, 0)):
    controller.setMotorsAngleInFixedTimestep(walk.walkAnglesStartRight[0], actionTime, 0)

for i in range(2):
    #Left foot step
    for i in range(np.size(walk.walkAnglesWalkingLeft, 0)):
        controller.setMotorsAngleInFixedTimestep(walk.walkAnglesWalkingLeft[i], actionTime, 0)
        
    for i in range(np.size(walk.walkAnglesWalkingRight, 0)):
        controller.setMotorsAngleInFixedTimestep(walk.walkAnglesWalkingRight[i], actionTime, 0)
        
#end of the walking. left
for i in range(np.size(walk.walkAnglesEndLeft, 0)):
    controller.setMotorsAngleInFixedTimestep(walk.walkAnglesEndLeft[i], actionTime, 0)
    
# rest for 2 second
waitTime = 2
repeatTime = int(waitTime / fixedTimeStep)
for _ in range(repeatTime):
    p.stepSimulation